In [2]:
import pandas as pd
import nltk.data
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alyonka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
df = pd.read_csv('cooked_dataset.csv', index_col='Unnamed: 0').reset_index(drop=True)
df.tail()

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count
32763,В суд передали дело еще четырех фигурантов «бо...,1,RuGPT3_small,Новостная статья: В суд передали дело еще четы...,в суд передано дело ещё четырех обвиняемых по ...,172.166667,4.860000,2562,312,37
32764,Бразильский футболист попросил у Кадырова росс...,1,RuGPT3_small,Новостная статья: Бразильский футболист попрос...,и на этот раз не отказался от него в россии в ...,56.937500,2.615142,1113,159,61
32765,«ВКонтакте» выкупила у Дурова дата-центр,1,RuGPT3_small,Новостная статья: «ВКонтакте» выкупила у Дуров...,в лыткарино в состоянии банкротства вконтакте ...,123.562500,2.657534,1197,160,36
32766,Захарова доверилась любви в деле восстановлени...,1,RuGPT3_small,Новостная статья: Захарова доверилась любви в ...,оригинал взят у в захаровой доверились любви в...,80.636364,2.339568,966,147,54
32767,Женщина залезла в канализацию ради спасения утят,1,RuGPT3_small,Новостная статья: Женщина залезла в канализаци...,в новосибирске в одном из отелей где проходили...,128.312500,2.472000,1283,196,32


In [12]:
df[df.text == 'Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.'].index

Index([], dtype='int64')

In [17]:
def remove_news(text):
    return text.replace('Новостная статья:', '').replace('*Примечание: В данной статье приведены гипотетические данные и информация.*', '')

def remove_between_stars(text):
    return re.sub(r'\*\*(.*?)\*\*', '', text)

def remove_title(row):
    title = row['title']
    text = row['text']
    index =text.find(title)
    if index != -1:
        return remove_news(text[:index] + text[index + len(title):]).replace('*', '').replace('\n+', ' ')
    else:
        return remove_news(remove_between_stars(text)).replace('*', '').replace('\n+', ' ')

def remove_non_alphanumeric(text):
    return re.sub(r'[^а-яА-ЯёЁa-zA-Z0-9.,\s]', '', text)

def remove_html_entities(text):
    entity_pattern = re.compile(r'&[a-zA-Z]+;')
    cleaned_text = re.sub(entity_pattern, ' ', text)
    return cleaned_text

df['text_without_title'] = df.apply(remove_title, axis=1)
df['text_without_title'] = df['text_without_title'].apply(remove_html_entities)
df['text_without_title'] = df['text_without_title'].str.replace('\n', ' ')
df['text_without_title'] = df['text_without_title'].str.replace('\t', ' ')
df['text_without_title'] = df['text_without_title'].apply(remove_non_alphanumeric)
df.tail()

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count,text_without_title
32763,В суд передали дело еще четырех фигурантов «бо...,1,RuGPT3_small,Новостная статья: В суд передали дело еще четы...,в суд передано дело ещё четырех обвиняемых по ...,172.166667,4.860000,2562,312,37,В суд передано дело ещё четырех обвиняемых п...
32764,Бразильский футболист попросил у Кадырова росс...,1,RuGPT3_small,Новостная статья: Бразильский футболист попрос...,и на этот раз не отказался от него в россии в ...,56.937500,2.615142,1113,159,61,и на этот раз не отказался от него В России...
32765,«ВКонтакте» выкупила у Дурова дата-центр,1,RuGPT3_small,Новостная статья: «ВКонтакте» выкупила у Дуров...,в лыткарино в состоянии банкротства вконтакте ...,123.562500,2.657534,1197,160,36,в Лыткарино В состоянии банкротства Вконтак...
32766,Захарова доверилась любви в деле восстановлени...,1,RuGPT3_small,Новостная статья: Захарова доверилась любви в ...,оригинал взят у в захаровой доверились любви в...,80.636364,2.339568,966,147,54,Оригинал взят у в Захаровой доверились люб...
32767,Женщина залезла в канализацию ради спасения утят,1,RuGPT3_small,Новостная статья: Женщина залезла в канализаци...,в новосибирске в одном из отелей где проходили...,128.312500,2.472000,1283,196,32,"В Новосибирске в одном из отелей, где прохо..."


In [18]:
# Загрузка токенизатора для русского языка
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')


def split_text_into_sentences(df):
    new_rows = []
    for index, row in df.iterrows():
        title = row['title']
        text = row['text_without_title']
        sentences = tokenizer.tokenize(text)
        model = row['model']
        generated = row['generated']
        for i in range(0, len(sentences), 3):  
            new_text = ' '.join(sentences[i:i+3])  
            new_rows.append({'text': new_text, 'model': model, 'generated': generated, 'title': title})
    return pd.DataFrame(new_rows)

new_df = split_text_into_sentences(df)


In [49]:
new_df = new_df.drop(new_df[new_df.text == 'А. б., ааб. я, т., е, я , я т., я, тв., яя, м. т..   Ао, б, о, г. Я, ср, и я. А, в, ю, н, ж. О, д, р. р., тъ. н. й. м, к. ю. д. з.'].index)

In [50]:
new_df[(new_df['model'] == 'RuGPT3_small')]

,text,model,generated,title
116650,", он будет представлен широкой публике в март...",RuGPT3_small,1,Анонсирован выпуск необычного секс-робота в 20...
116651,"Я не верю, что это будет секс . . автор видео ...",RuGPT3_small,1,Анонсирован выпуск необычного секс-робота в 20...
116652,"Я думаю, вы поняли, о чём речь. , ldi it 1. 2....",RuGPT3_small,1,Анонсирован выпуск необычного секс-робота в 20...
116654,Китайцы и белорусские автопроизводители дог...,RuGPT3_small,1,Китайцы договорились о сборке автомобилей Geel...
116655,"Учитывая, что Белоруссия является для автомо...",RuGPT3_small,1,Китайцы договорились о сборке автомобилей Geel...
...,...,...,...,...
132868,Датацентр Дарьялrdqu Москва 1400 кв.м. в ...,RuGPT3_small,1,«ВКонтакте» выкупила у Дурова дата-центр
132869,Ссылка на сайт httpwww.daradal.ru httpdadaralm...,RuGPT3_small,1,«ВКонтакте» выкупила у Дурова дата-центр
132871,Оригинал взят у в Захаровой доверились люб...,RuGPT3_small,1,Захарова доверилась любви в деле восстановлени...
132878,"В Новосибирске в одном из отелей, где прохо...",RuGPT3_small,1,Женщина залезла в канализацию ради спасения утят


In [44]:
new_df = new_df.drop(new_df[(new_df['model'] == 'RuGPT3_small')].index)

In [41]:
new_df

,text,model,generated,title
0,Популярный исполнитель Канье Уэст рассказал в ...,Human,0,Канье Уэст получил пожизненную подписку на Por...
1,"Однако Уэст заявил, что его взгляды на объекти...",Human,0,Канье Уэст получил пожизненную подписку на Por...
2,Мы хотим выразить нашу признательность господи...,Human,0,Канье Уэст получил пожизненную подписку на Por...
3,Такой подарок привел в восторг пользователей с...,Human,0,Канье Уэст получил пожизненную подписку на Por...
4,"Некоторые в шутливой форме уточняли, нет ли у ...",Human,0,Канье Уэст получил пожизненную подписку на Por...
...,...,...,...,...
132868,Датацентр Дарьялrdqu Москва 1400 кв.м. в ...,RuGPT3_small,1,«ВКонтакте» выкупила у Дурова дата-центр
132869,Ссылка на сайт httpwww.daradal.ru httpdadaralm...,RuGPT3_small,1,«ВКонтакте» выкупила у Дурова дата-центр
132871,Оригинал взят у в Захаровой доверились люб...,RuGPT3_small,1,Захарова доверилась любви в деле восстановлени...
132878,"В Новосибирске в одном из отелей, где прохо...",RuGPT3_small,1,Женщина залезла в канализацию ради спасения утят


In [51]:
new_df.to_csv('cooked_dataset_cutted.csv')

In [4]:
df_cutted = pd.read_csv('cooked_dataset_cutted.csv', index_col='Unnamed: 0').reset_index(drop=True)
df_cutted.tail()

,text,model,generated,title
132278,Датацентр Дарьялrdqu Москва 1400 кв.м. в ...,RuGPT3_small,1,«ВКонтакте» выкупила у Дурова дата-центр
132279,Ссылка на сайт httpwww.daradal.ru httpdadaralm...,RuGPT3_small,1,«ВКонтакте» выкупила у Дурова дата-центр
132280,Оригинал взят у в Захаровой доверились люб...,RuGPT3_small,1,Захарова доверилась любви в деле восстановлени...
132281,"В Новосибирске в одном из отелей, где прохо...",RuGPT3_small,1,Женщина залезла в канализацию ради спасения утят
132282,На Первом канале . А39А .В ласти Новосибирск...,RuGPT3_small,1,Женщина залезла в канализацию ради спасения утят


In [5]:
df_cutted = df_cutted.drop(df_cutted[df_cutted.model == 'RuGPT3_small'].index)
df_cutted.tail()

,text,model,generated,title
116531,"Мы должны помнить, что это не только проблема ...",GigaChat,1,Обама увидел в скандале с «Оскарами» более шир...
116532,Объявлены номинанты на кинопремию Ника В этом...,GigaChat,1,Объявлены номинанты на кинопремию «Ника»
116533,"В этой категории представлены такие фильмы, ка...",GigaChat,1,Объявлены номинанты на кинопремию «Ника»
116534,В номинации Лучший режиссер представлены Сэм М...,GigaChat,1,Объявлены номинанты на кинопремию «Ника»
116535,Она была основана в 1988 году и с тех пор ежег...,GigaChat,1,Объявлены номинанты на кинопремию «Ника»


In [8]:
df_cutted[df_cutted.model!='Human']

,text,model,generated,title
64855,"Сегодня в жилом доме, расположенном в москов...",YandexGPT,1,Пожар в жилом доме в Солнцево потушен
64856,В настоящий момент специалисты выясняют причин...,YandexGPT,1,Пожар в жилом доме в Солнцево потушен
64857,"Жители дома были эвакуированы, их жизни и здор...",YandexGPT,1,Пожар в жилом доме в Солнцево потушен
64858,Генеральная прокуратура России провела прове...,YandexGPT,1,В Генпрокуратуре заявили о наличии обманутых д...
64859,В результате этого люди оставались без жилья и...,YandexGPT,1,В Генпрокуратуре заявили о наличии обманутых д...
...,...,...,...,...
116531,"Мы должны помнить, что это не только проблема ...",GigaChat,1,Обама увидел в скандале с «Оскарами» более шир...
116532,Объявлены номинанты на кинопремию Ника В этом...,GigaChat,1,Объявлены номинанты на кинопремию «Ника»
116533,"В этой категории представлены такие фильмы, ка...",GigaChat,1,Объявлены номинанты на кинопремию «Ника»
116534,В номинации Лучший режиссер представлены Сэм М...,GigaChat,1,Объявлены номинанты на кинопремию «Ника»


In [9]:
df_cutted.to_csv('cooked_dataset_cutted_smaller.csv')